# Module 4: NLP and the Pipeline

* DS 6001
* Raf Alvarado

# Overview

We import a collection of texts and convert to F2. Then we annotate the collection to create an F3-level model.

# Set Up

## Configs

In [1]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']
epub_dir = 'epubs'

## Imports

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import re
import nltk

In [3]:
%matplotlib inline

## Import NLTK and download resources

If you need to install NLTK, see the [instructions here](https://www.nltk.org/install.html). You can also install this with Anaconda, like so:

`conda install nltk`

Once you have installed NLTK, you will need to download resources, which will happen when you run the following cell. If the interactive window opens, you may need to set your NLTK Data Directory, as described in the [instructions here](https://www.nltk.org/data.html). To set the directory, click on the File menu and select Change Download Directory. For central installation, set this to `C:\nltk_data` (Windows),`/usr/local/share/nltk_data` (Mac), or `/usr/share/nltk_data` (Unix). 

> If you did not install the data to one of the above central locations, you will need to set the NLTK_DATA environment variable to specify the location of the data. (On a Windows machine, right click on “My Computer” then select Properties > Advanced > Environment Variables > User Variables > New...)

In [4]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('tagsets')

[nltk_data] Downloading package punkt to /Users/rca2t/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/rca2t/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/rca2t/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package tagsets to /Users/rca2t/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


True

# Acquire

We download the novels of Jane Austen and Herman Melville from Project Gutenberg. I actually used a utility I created called PGTK:

* https://github.com/ontoligent-design/pgtk

# Inspect

Since Project Gutenberg texts vary widely in their markup, we define our chunking patterns by hand.

In [5]:
roman = '[IVXLCM]+'
caps = "[A-Z';, -]+"
chap_pats = {
    158: {
        'start_line': 37,
        'end_line': 16261,
        'volume': re.compile('^\s*VOLUME\s+{}\s*$'.format(roman)),
        'chapter': re.compile('^\s*CHAPTER\s+{}\s*$'.format(roman))
    },
    946: {
        'start_line': 38,
        'end_line': 2556,
        'chapter': re.compile("^\s*{}\s*$".format(roman))
    },
    1212: {
        'start_line': 77,
        'end_line': 3432,
        'chapter': re.compile("^\s*LETTER .* to .*$")
    },
    141: {\
        'start_line': 40,
        'end_line': 15376,
        'chapter': re.compile("^CHAPTER\s+{}$".format(roman))
    },
    121: {
        'start_line': 57,
        'end_line': 7874,
        'chapter': re.compile("^CHAPTER\s+\d+$")
    },
    105: {
        'start_line': 48,
        'end_line': 8360,
        'chapter': re.compile("^Chapter\s+\d+$")
    },
    1342: {
        'start_line': 37,
        'end_line': 13061,
        'chapter': re.compile("^Chapter\s+\d+$")
    },
    161: {
        'start_line': 43,
        'end_line': 12654,
        'chapter': re.compile("^CHAPTER\s+\d+$")          
    },
    15422: {
        'start_line': 193,
        'end_line': 7501,
        'chapter': re.compile("^\s*CHAPTER\s+{}\.".format(roman))
    },
    13720: {
        'start_line': 187,
        'end_line': 11470,
        'chapter': re.compile("^\s*CHAPTER\s+{}\s*$".format(roman))
    },
    13721: {
        'start_line': 164,
        'end_line': 13135,
        'chapter': re.compile("^\s*CHAPTER\s+{}\s*$".format(roman))
    },
    2701: {
        'start_line': 52,
        'end_line': 21743,
        'chapter': re.compile("^(ETYMOLOGY|EXTRACTS|CHAPTER)")
    },
    4045: {
        'start_line': 138,
        'end_line': 11655,
        'volume': re.compile("^\s*PART\s+{}\s*$".format(roman)),
        'chapter': re.compile("^\s*CHAPTER\s+{}\.\s*$".format(roman))
    },
    34970: {
        'start_line': 234,
        'end_line': 16267,
        'volume': re.compile("^\s*BOOK\s+{}\.\s*$".format(roman)),
        'chapter': re.compile("^\s*{}\.\s*$".format(roman))
    },
    8118: {
        'start_line': 142,
        'end_line': 12300,
        'chapter': re.compile("^\s*{}\. .*$".format(roman))
    },
    53861: {
        'start_line': 129,
        'end_line': 6904,
        'chapter': re.compile('^\s*{}\s*$'.format(caps))
    },
    21816: {
        'start_line': 309,
        'end_line': 11023,
        'chapter': re.compile('^CHAPTER\s+{}\.?$'.format(roman))
    },
    15859: {
        'start_line': 77,
        'end_line': 8619,
        'chapter': re.compile('^\s*{}\s*$'.format(caps))
    },
    1900: {
        'start_line': 43,
        'end_line': 11216,
        'chapter': re.compile("^CHAPTER\s+\w+\s*$")
    },
    10712: {
        'start_line': 205,
        'end_line': 15487,
        'chapter': re.compile("^CHAPTER\s+{}\.\s*$".format(roman))
    }
}

# Register and Chunk

In [6]:
def acquire_epubs(epub_list, chap_pats, OHCO=OHCO):
    
    my_lib = []
    my_doc = []

    for epub_file in epub_list:
        
        # Get PG ID from filename
        book_id = int(epub_file.split('-')[-1].split('.')[0].replace('pg',''))
        print("BOOK ID", book_id)
        
        # Import file as lines
        lines = open(epub_file, 'r', encoding='utf-8-sig').readlines()
        df = pd.DataFrame(lines, columns=['line_str'])
        df.index.name = 'line_num'
        df.line_str = df.line_str.str.strip()
        df['book_id'] = book_id
        
        # FIX CHARACTERS TO IMPROVE TOKENIZATION
        df.line_str = df.line_str.str.replace('—', ' — ')
        df.line_str = df.line_str.str.replace('-', ' - ')
        
        # Get book title and put into LIB table -- note problems, though
        book_title = re.sub(r"The Project Gutenberg eBook( of|,) ", "", df.loc[0].line_str, flags=re.IGNORECASE)
        book_title = re.sub(r"Project Gutenberg's ", "", book_title, flags=re.IGNORECASE)
        
        # Remove cruft
        a = chap_pats[book_id]['start_line'] - 1
        b = chap_pats[book_id]['end_line'] + 1
        df = df.iloc[a:b]
        
        # Chunk by chapter
        chap_lines = df.line_str.str.match(chap_pats[book_id]['chapter'])
        chap_nums = [i+1 for i in range(df.loc[chap_lines].shape[0])]
        df.loc[chap_lines, 'chap_num'] = chap_nums
        df.chap_num = df.chap_num.ffill()

        # Clean up
        df = df[~df.chap_num.isna()] # Remove chapter heading lines
        df = df.loc[~chap_lines] # Remove everything before Chapter 1
        df['chap_num'] = df['chap_num'].astype('int')
        
        # Group -- Note that we exclude the book level in the OHCO at this point
        df = df.groupby(OHCO[1:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string
        
        # Split into paragrpahs
        df = df['line_str'].str.split(r'\n\n+', expand=True).stack().to_frame().rename(columns={0:'para_str'})
        df.index.names = OHCO[1:3] # MAY NOT BE NECESSARY UNTIL THE END
        df['para_str'] = df['para_str'].str.replace(r'\n', ' ').str.strip()
        df = df[~df['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs
        
        # Set index
        df['book_id'] = book_id
        df = df.reset_index().set_index(OHCO[:3])

        # Register
        my_lib.append((book_id, book_title, epub_file))
        my_doc.append(df)

    docs = pd.concat(my_doc)
    library = pd.DataFrame(my_lib, columns=['book_id', 'book_title', 'book_file']).set_index('book_id')
    return library, docs

In [7]:
epubs = [epub for epub in sorted(glob(epub_dir+'/*.txt'))]
LIB, DOC = acquire_epubs(epubs, chap_pats)

BOOK ID 158
BOOK ID 946
BOOK ID 1212
BOOK ID 141
BOOK ID 121
BOOK ID 105
BOOK ID 1342
BOOK ID 161
BOOK ID 15422
BOOK ID 13720
BOOK ID 13721
BOOK ID 2701
BOOK ID 4045
BOOK ID 34970
BOOK ID 8118
BOOK ID 53861
BOOK ID 21816
BOOK ID 15859
BOOK ID 1900
BOOK ID 10712


In [8]:
DOC.sample(10)

,,,para_str
book_id,chap_num,para_num,
141,37,0,"Mr. Crawford gone, Sir Thomas's next object wa..."
21816,18,2,"""But do you think it the fair thing to unmask ..."
13720,46,13,"So away floated the Chamois, like a vagrant cl..."
13721,17,54,"""Well said, old man,"" cried Babbalanja; ""for, ..."
2701,13,7,If there yet lurked any ice of indifference to...
1342,43,14,"""Does that young lady know Mr. Darcy?"""
13721,76,116,"BABBALANJA - - Very funny, your Highness: - ..."
141,42,15,"After a moment's reflection, Mr. Crawford repl..."
121,16,30,"Isabella shrugged her shoulders and smiled, th..."


# Tokenize and Annotate

We use NLTK this time. Note that this process takes some time, mainly because the NLTK functions are not optimized for dataframes.

In [9]:
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):
    
    # Paragraphs to Sentences
    df = doc_df.para_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pd.Series(nltk.pos_tag(nltk.word_tokenize(x)))
        return s
            
    df = df.sent_str\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [10]:
TOKEN = tokenize(DOC, ws=True)

In [11]:
TOKEN.head()

pos_tuple  pos  \
book_id chap_num para_num sent_num token_num                           
158     1        1        0        0                (Emma, NNP)  NNP   
                                   1          (Woodhouse,, NNP)  NNP   
                                   2            (handsome,, NN)   NN   
                                   3              (clever,, NN)   NN   
                                   4                  (and, CC)   CC   

                                               token_str  
book_id chap_num para_num sent_num token_num              
158     1        1        0        0                Emma  
                                   1          Woodhouse,  
                                   2           handsome,  
                                   3             clever,  
                                   4                 and

In [12]:
TOKEN[TOKEN.pos.str.match('^JJ')]

pos_tuple  pos  \
book_id chap_num para_num sent_num token_num                             
158     1        1        0        8            (comfortable, JJ)   JJ   
                                   11                 (happy, JJ)   JJ   
                                   19                 (best, JJS)  JJS   
                                   27                (twenty, JJ)   JJ   
                                   36                (little, JJ)   JJ   
                 2        0        3              (youngest, JJS)  JJS   
                                   11         (affectionate,, JJ)   JJ   
                                   30                 (early, JJ)   JJ   
                          1        11                 (more, JJR)  JJR   
                                   14            (indistinct, JJ)   JJ   
                                   27             (excellent, JJ)   JJ   
                                   34                (little, JJ)   JJ   
                                   35                 (short, JJ)   JJ   
                 3        0        0                (Sixteen, JJ)   JJ   
                                   16               (friend,, JJ)   JJ   
                          2        9                (nominal, JJ)   JJ   
                                   47             (attached,, JJ)   JJ   
                 4        0        1                   (real, JJ)   JJ   
                                   2                 (evils,, JJ)   JJ   
                                   14                  (much, JJ)   JJ   
                                   16                   (own, JJ)   JJ   
                                   24                (little, JJ)   JJ   
                                   38                  (many, JJ)   JJ   
                          1        5                (present, JJ)   JJ   
                                   7           (unperceived,, JJ)   JJ   
                 5        0        5                 (gentle, JJ)   JJ   
                                   18          (disagreeable, JJ)   JJ   
                          3        16              (mournful, JJ)   JJ   
                          4        5                  (bride, JJ)   JJ   
                                   23                 (third, JJ)   JJ   
...                                                           ...  ...   
10712   92       17       0        15              (unhappy,, JJ)   JJ   
                          2        6                (gallant, JJ)   JJ   
                                   11                   (top, JJ)   JJ   
                 18       0        10                  (main, JJ)   JJ   
                                   34                   (own, JJ)   JJ   
                          1        21           (unlegalised, JJ)   JJ   
                 19       0        27                (little, JJ)   JJ   
                          1        17                 (clean, JJ)   JJ   
                                   22                   (oft, JJ)   JJ   
                                   24               (painted, JJ)   JJ   
                                   32                  (vast, JJ)   JJ   
                 20       0        8                    (up,, JJ)   JJ   
                                   21                 (more, JJR)  JJR   
                                   35                   (tow, JJ)   JJ   
                                   40                   (own, JJ)   JJ   
                 21       0        15          (subterranean, JJ)   JJ   
                 22       1        4                   (many, JJ)   JJ   
                          2        5                   (full, JJ)   JJ   
                          3        25            (indefinite, JJ)   JJ   
                          4        2                 (worst, JJS)  JJS   
                          5        2                   (last, JJ)   JJ   
                                   8               (another;

# Reduce

Extract a vocabulary from the TOKEN table

In [13]:
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')

In [14]:
VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [15]:
VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')

In [16]:
VOCAB

,term_str,n,num
term_id,,,
0,,50493,0
1,0,2,1
2,1,18,1
3,10,6,1
4,100,2,1
5,1000,2,1
6,10000,3,1
7,1000000,1,1
8,10000000,1,1


# Annotate (VOCAB)

## Add Stopwords

We use NLTK's built in stopword list for English. Note that we can add and subtract from this list, or just create our own list and keep it in our data model.

In [17]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [18]:
sw.sample(10)

,dummy
term_str,
where,1
not,1
it,1
up,1
on,1
them,1
ain,1
won't,1
nor,1


In [19]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [20]:
VOCAB[VOCAB.stop == 1].sample(10)

,term_str,n,num,stop
term_id,,,,
11261,during,535,0,1
11324,each,981,0,1
16811,herself,1540,0,1
39745,whom,860,0,1
35891,to,56220,0,1
39382,we,6291,0,1
2121,as,17786,0,1
35380,than,3949,0,1
24367,off,1675,0,1


## Add Stems

In [21]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
VOCAB['p_stem'] = VOCAB.term_str.apply(stemmer.stem)

In [22]:
VOCAB.sample(10)

,term_str,n,num,stop,p_stem
term_id,,,,,
20587,legged,33,0,0,leg
23602,naughtiness,1,0,0,naughti
23103,morose,5,0,0,moros
27337,prescriptive,1,0,0,prescript
15833,grooms,3,0,0,groom
21501,magnetical,1,0,0,magnet
5008,cadell,1,0,0,cadel
1859,arbored,3,0,0,arbor
30447,sachem,1,0,0,sachem


# Save

In [23]:
DOC.to_csv('DOC.csv')
LIB.to_csv('LIB.csv')
VOCAB.to_csv('VOCAB.csv')
TOKEN.to_csv('TOKEN.csv')

# Save to SQLite

In [24]:
import sqlite3

TOKEN = TOKEN.drop('pos_tuple', 1)

with sqlite3.connect('mod4-corpus.db') as db:
    DOC.to_sql('doc', db, index=True, if_exists='replace')
    LIB.to_sql('lib', db, index=True, if_exists='replace')
    VOCAB.to_sql('vocab', db, index=True, if_exists='replace')
    TOKEN.to_sql('token', db, index=True, if_exists='replace')

# Appendix: Explore NER tagging

In [25]:
%time NER = TOKEN.groupby(OHCO[:4]).pos_tuple.apply(lambda x: nltk.ne_chunk(x.tolist())).to_frame().rename(columns={'pos_tuple':'nltk_sentence'})

AttributeError: 'DataFrameGroupBy' object has no attribute 'pos_tuple'

In [26]:
NER.iloc[8].values[0]

NameError: name 'NER' is not defined

In [ ]:
NER.iloc[26].values[0]

# POS Tagset

This a token-level feature -- not a vocab feature

In [ ]:
nltk.help.upenn_tagset()